In [1]:
# 記事反映
import requests
import json
from wordpress_xmlrpc.methods import media
from oauth2client.service_account import ServiceAccountCredentials
import gspread
from wordpress_xmlrpc.methods.posts import GetPosts, NewPost
from wordpress_xmlrpc.methods.users import GetUserInfo
from wordpress_xmlrpc.methods import media, posts
from wordpress_xmlrpc.compat import xmlrpc_client
from wordpress_xmlrpc import Client, WordPressPost
from PIL import Image, ImageDraw, ImageFont
from dateutil.relativedelta import relativedelta
import matplotlib.pyplot as plt
import os
from selenium import webdriver
import time
import pandas as pd
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.support.ui import Select
import urllib
from bs4 import BeautifulSoup
import re
import csv
import codecs
import urllib.request as req
import glob
from matplotlib.ticker import MaxNLocator
import matplotlib.ticker as ticker
from matplotlib import rcParams
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from datetime import timedelta
import datetime
from dotenv import load_dotenv
from selenium.webdriver.common.by import By

from sqlalchemy import (Column, String, Text, ForeignKey, \
                create_engine, MetaData, DECIMAL, DATETIME, exc, event, Index)
from sqlalchemy.schema import UniqueConstraint
from sqlalchemy.orm import (sessionmaker, relationship, scoped_session)
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.dialects.mysql import INTEGER as Integer
from datetime import datetime
from sqlalchemy import Column, Integer, create_engine, MetaData, DATETIME, String,Date,Float
from sqlalchemy.orm import sessionmaker
from pathlib import Path
import datetime
from sqlalchemy import create_engine, text
from  src.utils import *
print('ライブラリの読み込み完了')

# .envファイルの内容を読み込見込む
load_dotenv()

def login_scraping_site(area_name):
    global browser
    options = Options()
    #options.add_argument('--headless')
    options.add_argument("--no-sandbox")
    browser = webdriver.Chrome(ChromeDriverManager().install(),options=options)#ChromeDriverManager().install()
    browser.implicitly_wait(10)
    url_login = f"https://{os.getenv('SCRAPING_SYUZAI_DOMAIN')}/login_form_mail"
    #admageを開く
    browser.get(url_login)
    browser.implicitly_wait(10)
    # id
    element = browser.find_element(By.NAME,"id")
    element.click()
    element.clear()
    browser.implicitly_wait(10)
    element.send_keys(os.getenv('REPORT_SITE_ID'))
    # pw
    element = browser.find_element(By.NAME,"pass")
    element.click()
    element.clear()
    browser.implicitly_wait(10)
    element.send_keys(os.getenv('REPORT_SITE_PW'))
    browser.implicitly_wait(10)
    element = browser.find_element(By.CLASS_NAME,"box_hole_view_report_input")
    element.click()
    browser.implicitly_wait(10)
    url = f"https://{os.getenv('SCRAPING_SYUZAI_DOMAIN')}/select_area"
    browser.get(url)
    browser.implicitly_wait(10)
    url = f"https://{os.getenv('SCRAPING_SYUZAI_DOMAIN')}/?area={area_name}"
    browser.get(url)
    browser.implicitly_wait(10)
    return browser


browser = login_scraping_site('chubu')
#エリアの都道府県の名前と対応番号を取得
prefectures_box_elements = browser.find_element(By.CLASS_NAME,"one_sp_block")
prefectures_elements:list = prefectures_box_elements.find_elements(By.CLASS_NAME,"mgn_serch_list_bottom")
prefecture_name_and_number_dict:dict[str:str] = {}

for element in prefectures_elements:
    prefecture_name:str = element.find_element(By.CLASS_NAME,"side_bar_L_O").text
    #print(prefecture_name)
    link_url:str = element.find_element(By.TAG_NAME,'a').get_attribute('href')
    prefecture_number:int = link_url.split('ken=')[-1]
    #print(prefecture_number)
    prefecture_name_and_number_dict[prefecture_name] = prefecture_number
prefecture_name_and_number_dict

読み込み完了
ライブラリの読み込み完了


C:\Users\tsc95\AppData\Local\Temp\ipykernel_48568\2930225511.py:61: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(ChromeDriverManager().install(),options=options)#ChromeDriverManager().install()


{'愛知県': '23',
 '静岡県': '22',
 '山梨県': '19',
 '長野県': '20',
 '岐阜県': '21',
 '新潟県': '15',
 '富山県': '16',
 '石川県': '17',
 '福井県': '18',
 '三重県': '24'}

In [2]:
i = 0
#取材取得日指定
target_date :datetime = datetime.date.today() + datetime.timedelta(days=1)
target_date_str_sql:str = target_date.strftime('%Y-%m-%d')
#合成用の空のDF
furture_syuzai_list_df = pd.DataFrame(index=[], columns=['都道府県','イベント日','店舗名','取材名','取材ランク'])


for prefecture_name,prefecture_number in prefecture_name_and_number_dict.items():
    print(prefecture_name)
    url = f"https://{os.getenv('SCRAPING_SYUZAI_DOMAIN')}/osusume_list?ken={prefecture_number}&ymd={target_date}"
    browser.get(url)
    browser.implicitly_wait(10)
    kiji_element_box = browser.find_element(By.CLASS_NAME,"osusume_list_container")
    kiji_elements_list:list = kiji_element_box.find_elements(By.CLASS_NAME,"osbox")
    for i,syuzai_record_element in enumerate(kiji_elements_list):
        if 'プレミアム会員登録' == browser.find_element(By.CLASS_NAME,"menu_child").text:
            browser = login_scraping_site('chubu')
            url = f"https://{os.getenv('SCRAPING_SYUZAI_DOMAIN')}/osusume_list?ken={prefecture_number}&ymd={target_date}"
            browser.get(url)
            browser.implicitly_wait(10)
        else:
            pass
        #print(syuzai_record_element.text)
        syuzai_record_element_list = syuzai_record_element.find_elements(By.CLASS_NAME,"list_event_name")
        browser.implicitly_wait(1)
        print(syuzai_record_element_list)
        if len(syuzai_record_element_list) == 0:
            continue
        tenpo_name = syuzai_record_element.find_element(By.CLASS_NAME,"oslh2").text
        for syuzai_name_element in syuzai_record_element_list:
            #print(syuzai_name_element.text)
            syuzai_rank = syuzai_name_element.find_element(By.CLASS_NAME,"list_event_name_rank").text
            schedule_name = syuzai_name_element.find_element(By.CLASS_NAME,"list_event_name_li").text
            print(tenpo_name,syuzai_rank,schedule_name)
            record = pd.Series([prefecture_name,target_date, tenpo_name,schedule_name,syuzai_rank], index=furture_syuzai_list_df.columns)
            furture_syuzai_list_df = furture_syuzai_list_df.append(record, ignore_index=True)
    break
#browser.quit()


愛知県
[<selenium.webdriver.remote.webelement.WebElement (session="8bf3300b9767d020885fce2461fd9e40", element="ccb5d383-9da1-41d2-aa8f-ec465e58c2de")>, <selenium.webdriver.remote.webelement.WebElement (session="8bf3300b9767d020885fce2461fd9e40", element="57835b05-da92-4c48-b7dd-7157850fc9dd")>, <selenium.webdriver.remote.webelement.WebElement (session="8bf3300b9767d020885fce2461fd9e40", element="4fe6b42f-1911-4341-86b3-5ce316a4cca6")>, <selenium.webdriver.remote.webelement.WebElement (session="8bf3300b9767d020885fce2461fd9e40", element="ee8df490-9096-4e96-bf3b-4450df3e2670")>]
キクヤ春日井店 S ナビ子AI予想【差枚プラス】★5 (毎週 金曜日)
キクヤ春日井店 S ナビ子AI予想【差枚プラス】★3 (4のつく日)


C:\Users\tsc95\AppData\Local\Temp\ipykernel_48568\3907992567.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  furture_syuzai_list_df = furture_syuzai_list_df.append(record, ignore_index=True)
C:\Users\tsc95\AppData\Local\Temp\ipykernel_48568\3907992567.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  furture_syuzai_list_df = furture_syuzai_list_df.append(record, ignore_index=True)
C:\Users\tsc95\AppData\Local\Temp\ipykernel_48568\3907992567.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  furture_syuzai_list_df = furture_syuzai_list_df.append(record, ignore_index=True)
C:\Users\tsc95\AppData\Local\Temp\ipykernel_48568\3907992567.py:37: FutureWarning: The frame.append method is deprecated and will be removed from

キクヤ春日井店 S ナビ子AI予想【機種仕掛け】★4 (毎週 金曜日)
キクヤ春日井店 ・ シーサーペントOVER
[<selenium.webdriver.remote.webelement.WebElement (session="8bf3300b9767d020885fce2461fd9e40", element="8e12b238-796a-4839-88cf-20714bbe680e")>]
キング観光 サウザンド栄若宮大通店 S オフミー開催(BBオフ会)
[<selenium.webdriver.remote.webelement.WebElement (session="8bf3300b9767d020885fce2461fd9e40", element="5cc4255f-c824-4d1d-97e8-5a21462459a6")>, <selenium.webdriver.remote.webelement.WebElement (session="8bf3300b9767d020885fce2461fd9e40", element="024fed3f-5107-4a28-8ef0-24563a569848")>]
ミカド一宮インター店 S ナビ子AI予想【差枚プラス】★5 (4のつく日)


C:\Users\tsc95\AppData\Local\Temp\ipykernel_48568\3907992567.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  furture_syuzai_list_df = furture_syuzai_list_df.append(record, ignore_index=True)
C:\Users\tsc95\AppData\Local\Temp\ipykernel_48568\3907992567.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  furture_syuzai_list_df = furture_syuzai_list_df.append(record, ignore_index=True)
C:\Users\tsc95\AppData\Local\Temp\ipykernel_48568\3907992567.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  furture_syuzai_list_df = furture_syuzai_list_df.append(record, ignore_index=True)
C:\Users\tsc95\AppData\Local\Temp\ipykernel_48568\3907992567.py:37: FutureWarning: The frame.append method is deprecated and will be removed from

ミカド一宮インター店 A ナビ子AI予想【機種仕掛け】★3 (4のつく日)
[<selenium.webdriver.remote.webelement.WebElement (session="8bf3300b9767d020885fce2461fd9e40", element="bdc2d3cd-a476-4912-8309-c006c7b41801")>, <selenium.webdriver.remote.webelement.WebElement (session="8bf3300b9767d020885fce2461fd9e40", element="25397a0f-8d24-455a-bf50-0355842c2e41")>]
KEIZ 尾張旭店 B ナビ子AI予想【機種仕掛け】★2 (4のつく日)
KEIZ 尾張旭店 C まなみん来店
[<selenium.webdriver.remote.webelement.WebElement (session="8bf3300b9767d020885fce2461fd9e40", element="21f08fb5-8945-4d7d-b415-a3c9778f12f5")>]
プレイランドキャッスル熱田店 A シーサ。オフミー取材


...

C:\Users\tsc95\AppData\Local\Temp\ipykernel_48568\3907992567.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  furture_syuzai_list_df = furture_syuzai_list_df.append(record, ignore_index=True)


[]
[]
[]
[]
[]
[<selenium.webdriver.remote.webelement.WebElement (session="8bf3300b9767d020885fce2461fd9e40", element="ecb0f527-4f9d-41b9-bf7b-e31b8c27d5d8")>]
マルハン東浦店 ・ 光るん×スロッター


C:\Users\tsc95\AppData\Local\Temp\ipykernel_48568\3907992567.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  furture_syuzai_list_df = furture_syuzai_list_df.append(record, ignore_index=True)


[]
[]
[<selenium.webdriver.remote.webelement.WebElement (session="8bf3300b9767d020885fce2461fd9e40", element="84f108b9-8977-4175-b7d2-181dceb0014f")>]
ZENT木曽川店 ・ Dynamite取材 暴走MISSION取材
[<selenium.webdriver.remote.webelement.WebElement (session="8bf3300b9767d020885fce2461fd9e40", element="b4944d46-a3d2-4089-9dce-ff4993fb1246")>]
コンコルド880愛西日比野駅前店 ・ 光るん×スロッター
[<selenium.webdriver.remote.webelement.WebElement (session="8bf3300b9767d020885fce2461fd9e40", element="438c76e0-fe66-4753-a9cc-0fd28ca4f035")>]
マルハン茶屋が坂店 ・ 光るん×スロッター


C:\Users\tsc95\AppData\Local\Temp\ipykernel_48568\3907992567.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  furture_syuzai_list_df = furture_syuzai_list_df.append(record, ignore_index=True)
C:\Users\tsc95\AppData\Local\Temp\ipykernel_48568\3907992567.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  furture_syuzai_list_df = furture_syuzai_list_df.append(record, ignore_index=True)
C:\Users\tsc95\AppData\Local\Temp\ipykernel_48568\3907992567.py:37: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  furture_syuzai_list_df = furture_syuzai_list_df.append(record, ignore_index=True)


In [3]:
twitter = UtilsTwitterClass()
print(vars(twitter))

username = os.getenv('DB_USER_NAME')
password=os.getenv('DB_PASSWORD')
host=os.getenv('DB_HOST')
port='3306'
database=os.getenv('DB_NAME')
db_url = f"mysql+mysqlconnector://{username}:{password}@{host}:{port}/{database}"

# DB接続するためのEngineインスタンス
engine = create_engine(db_url, echo=False)

# Sessionの作成
session = scoped_session(sessionmaker(autocommit = False,autoflush = False,bind = engine))
# modelで使用する
Base = declarative_base()
Base.query = session.query_property()





{'tomorrow_8numbers_string': '20230324', 'tomorrow_date_stinrg_jp': '3/24(金)', 'tomorrow_date_string_sql': '2023-03-24', 'yesterday': datetime.date(2023, 3, 22), 'tomorrow': datetime.date(2023, 3, 24), 'yesterday_date_string': '3/22(水)', 'id': '', 'pw': '', 'tweet_text': '', 'image_path_list': [], 'main_tweet_text': '', 'project_dir_path': 'C:\\Users\\tsc95\\Desktop\\syuzai_map_django_site', 'image_dir_path': 'C:\\Users\\tsc95\\Desktop\\syuzai_map_django_site\\image', 'tweet_footer_text': '\n\n#XXXX 20230322\n\n', 'created_image_dir_path': 'C:\\Users\\tsc95\\Desktop\\syuzai_map_django_site\\image\\image', 'servise_account_json_path': 'C:\\Users\\tsc95\\Desktop\\syuzai_map_django_site\\config\\twitteranalytics-jsonsercretkey.json'}


C:\Users\tsc95\AppData\Local\Temp\ipykernel_48568\1275897117.py:17: MovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [4]:
import mysql
import mysql.connector

cnx = mysql.connector.connect(
                        user = os.getenv('DB_USER_NAME'),
                        password=os.getenv('DB_PASSWORD'), 
                        host=os.getenv('DB_HOST'), 
                        port='3306',
                        database=os.getenv('DB_NAME'))

cursor = cnx.cursor()


In [5]:
sql = f"""
        SELECT *
        FROM pledge
        """
print(sql)
cursor.execute(sql)
#cols = [col[0] for col in cursorsor.description]
pledge_df = pd.DataFrame(cursor.fetchall(),columns = ['id','取材名','媒体名','公約内容','取得時間'])
display(pledge_df)
convert_parlar_name_df = pd.read_csv(r'csv\店舗名変換テーブル.csv')
convert_parlar_name_df = convert_parlar_name_df.rename(columns={'ホールナビ店舗名':'店舗名'})
convert_parlar_name_df


        SELECT *
        FROM pledge
        


,id,取材名,媒体名,公約内容,取得時間
0,1,ｙｔｒ来店実践,SEVENSTV,来店実践,2023-03-16 03:56:31
1,2,Ａレポート,アツ姫,Aタイプに高設定投入,2023-03-16 03:56:31
2,3,黒バラの極意(来店ver),ジャンバリ.TV,,2023-03-16 03:56:31
3,4,鮮魚レポート,アメスロ,,2023-03-16 03:56:31
4,5,魔法少女レポート スロット取材,爆ガチ！,まどかマギカシリーズが狙い目,2023-03-16 03:56:31
...,...,...,...,...,...
881,100130,旧イベ(毎月 第4日曜日),旧イベ,None,2023-03-22 23:51:55
882,100131,旧イベ(毎月 27日),旧イベ,None,2023-03-22 23:51:55
883,100132,旧イベ(毎月 28日),旧イベ,None,2023-03-22 23:51:55
884,100133,近隣の旧イベ(毎月 28日)対抗日,旧イベ,None,2023-03-22 23:51:55


,都道府県,店舗kintone,shop_id,店舗名,DMM店舗名,アナスロ店舗名,緯度,経度,住所
0,北海道,P01378,1378,アミューズメントパーラー吉兆,アミューズメントパーラー吉兆,アミューズメントパーラー吉兆,43.085828,141.514724,北海道江別市大麻泉町47-37
1,北海道,P01582,1582,ひまわり音更店,音更ひまわり,ひまわり音更店,42.961369,143.207331,北海道河東郡音更町木野大通東13丁目1番21
2,北海道,P01351,13160,EBH,EBH,EBH,43.106287,141.292820,北海道札幌市西区発寒15条4-13-1
3,茨城県,P02562,2562,キコーナ水戸駅南店,キコーナ水戸駅南店,キコーナ水戸駅南店,36.365522,140.477010,茨城県水戸市城南1-4-1
4,神奈川県,P04933,4933,ピーズ・タウン桜ヶ丘,P’Stown桜ヶ丘,ピーズ・タウン桜ヶ丘,35.451103,139.465162,神奈川県大和市福田1-6-10
...,...,...,...,...,...,...,...,...,...
2352,東京都,P00363,363,楽園渋谷駅前店,楽園渋谷駅前店,楽園渋谷駅前店,35.658285,139.699800,東京都渋谷区道玄坂1-3-1
2353,山梨県,P12714,13138,マルハン甲府店,マルハン甲府店,マルハン甲府店,35.631724,138.553588,山梨県甲府市国母8-20-10
2354,東京都,P12738,13170,メッセ西荻窪店,メッセ西荻窪店,メッセ西荻窪店,35.703498,139.600920,東京都杉並区西荻南3-14-7
2355,埼玉県,P12792,13227,マルキ戸田店,マルキ戸田店,マルキ戸田店,NaN,NaN,埼玉県戸田市美女木東1-3-2


In [6]:
parlar_name_list = list(furture_syuzai_list_df['店舗名'].unique())
print('parlar_name_list_len:',len(parlar_name_list))
# マーカープロット（ポップアップ設定，色変更，アイコン変更）
text = ''
tomorrow_schedule_df = pd.DataFrame(index=[], columns=[])
for parlar_name in parlar_name_list:
    query = f"select * from schedule WHERE 店舗名 = '{parlar_name}' AND イベント日 = '{target_date_str_sql}'"
    print(query)
    extract_parlar_name_df = pd.read_sql_query(sql=query, con=engine.connect())
    extract_parlar_name_df =  extract_parlar_name_df[~extract_parlar_name_df['取材名'].str.contains('ナビ子')]
    extract_parlar_name_df = extract_parlar_name_df[~extract_parlar_name_df.duplicated(subset=['イベント日','曜日','店舗名','取材名','媒体名'])]
    display(extract_parlar_name_df)
    tomorrow_schedule_df = pd.concat([tomorrow_schedule_df, extract_parlar_name_df])
    #display(convert_parlar_name_df)
    #tenpo_syuzai_merge_df = pd.merge(extract_parlar_name_df, convert_parlar_name_df, on='店舗名')
    # merge_df = pd.merge(tenpo_syuzai_merge_df, pledge_df, on='取材名')
    # merged_df =merge_df[merge_df['都道府県_x'] == '愛知県'].sort_values('取材ランク')
tomorrow_schedule_df
# tenpo_syuzai_merge_df = pd.merge(tomorrow_schedule_df, convert_parlar_name_df, on='店舗名')
# #tenpo_syuzai_merge_df
# merge_df = pd.merge(tomorrow_schedule_df,pledge_df, on='取材名')
# merge_df

parlar_name_list_len: 34
select * from schedule WHERE 店舗名 = 'キクヤ春日井店' AND イベント日 = '2023-03-24'


,id,都道府県,イベント日,曜日,店舗名,取材名,媒体名,取材ランク,取得時間
1,4768,愛知県,2023-03-24,金,キクヤ春日井店,シーサーペントOVER,アツ姫,・,2023-03-18 11:06:06


select * from schedule WHERE 店舗名 = 'キング観光 サウザンド栄若宮大通店' AND イベント日 = '2023-03-24'


,id,都道府県,イベント日,曜日,店舗名,取材名,媒体名,取材ランク,取得時間
0,4833,愛知県,2023-03-24,金,キング観光 サウザンド栄若宮大通店,オフミー開催(BBオフ会),オフミー,S,2023-03-18 11:06:06


select * from schedule WHERE 店舗名 = 'ミカド一宮インター店' AND イベント日 = '2023-03-24'


...

select * from schedule WHERE 店舗名 = 'コンコルド880愛西日比野駅前店' AND イベント日 = '2023-03-24'


,id,都道府県,イベント日,曜日,店舗名,取材名,媒体名,取材ランク,取得時間
0,3918,愛知県,2023-03-24,金,コンコルド880愛西日比野駅前店,光るん×スロッター,パチンコ店長のホール攻略,・,2023-03-17 11:05:46


select * from schedule WHERE 店舗名 = 'マルハン茶屋が坂店' AND イベント日 = '2023-03-24'


,id,都道府県,イベント日,曜日,店舗名,取材名,媒体名,取材ランク,取得時間
0,3919,情報なし,2023-03-24,金,マルハン茶屋が坂店,光るん×スロッター,パチンコ店長のホール攻略,・,2023-03-17 11:05:46


,id,都道府県,イベント日,曜日,店舗名,取材名,媒体名,取材ランク,取得時間
1,4768,愛知県,2023-03-24,金,キクヤ春日井店,シーサーペントOVER,アツ姫,・,2023-03-18 11:06:06
0,4833,愛知県,2023-03-24,金,キング観光 サウザンド栄若宮大通店,オフミー開催(BBオフ会),オフミー,S,2023-03-18 11:06:06
1,3985,愛知県,2023-03-24,金,ミカド一宮インター店,旧イベ(4のつく日),旧イベ,・,2023-03-17 11:05:46
0,3751,愛知県,2023-03-24,金,KEIZ 尾張旭店,まなみん来店,その他取材・企画,C,2023-03-17 11:05:46
0,4834,愛知県,2023-03-24,金,プレイランドキャッスル熱田店,シーサ。オフミー取材,オフミー,A,2023-03-18 11:06:06
0,3856,愛知県,2023-03-24,金,キング観光サウザンド名古屋駅柳橋店,スロパチステーション潜入取材,スロパチステーション,S,2023-03-17 11:05:46
0,3920,愛知県,2023-03-24,金,マルハン新守山駅前店,光るん×スロッター,パチンコ店長のホール攻略,・,2023-03-17 11:05:46
0,3744,愛知県,2023-03-24,金,メガコンコルド1020刈谷知立店,ズバスロ,あかだ主任のガチンコ…,・,2023-03-17 11:05:46
0,3901,愛知県,2023-03-24,金,ミカド稲沢店,マキシマムスロッター,パチンコ店長のホール攻略,B,2023-03-17 11:05:46
2,3986,愛知県,2023-03-24,金,ミカド稲沢店,旧イベ(4のつく日),旧イベ,・,2023-03-17 11:05:46


NameError: name 'merge_df' is not defined

In [26]:
merged_syuzai_pledge_df = pd.merge(furture_syuzai_list_df,pledge_df,how='left',on='取材名')
merged_syuzai_pledge_df = merged_syuzai_pledge_df[~merged_syuzai_pledge_df['取材名'].str.contains('ナビ子')]
merged_syuzai_pledge_df = merged_syuzai_pledge_df.fillna('未調査')
merged_syuzai_pledge_df = merged_syuzai_pledge_df.replace({'': '未調査'})
merged_syuzai_pledge_df
# def kouyaku(text):
#     if text == '':
#         text ='未調査'
#     else:
#         pass
# merged_syuzai_pledge_df['公約内容'] = merged_syuzai_pledge_df['公約内容'].map(kouyaku)


,都道府県,イベント日,店舗名,取材名,取材ランク,id,媒体名,公約内容,取得時間
3,愛知県,2023-03-24,キクヤ春日井店,シーサーペントOVER,・,481.0,アツ姫,未調査,2023-03-16 03:56:54
4,愛知県,2023-03-24,キング観光 サウザンド栄若宮大通店,オフミー開催(BBオフ会),S,536.0,オフミー,未調査,2023-03-16 03:56:58
8,愛知県,2023-03-24,KEIZ 尾張旭店,まなみん来店,C,579.0,その他取材・企画,未調査,2023-03-16 03:57:00
9,愛知県,2023-03-24,プレイランドキャッスル熱田店,シーサ。オフミー取材,A,483.0,オフミー,未調査,2023-03-16 03:56:54
10,愛知県,2023-03-24,キング観光サウザンド名古屋駅柳橋店,スロパチステーション潜入取材,S,432.0,スロパチステーション,未調査,2023-03-16 03:56:52
13,愛知県,2023-03-24,マルハン新守山駅前店,光るん×スロッター,・,198.0,パチンコ店長のホール攻略,未調査,2023-03-16 03:56:41
14,愛知県,2023-03-24,メガコンコルド1020刈谷知立店,ズバスロ,・,416.0,あかだ主任のガチンコ…,未調査,2023-03-16 03:56:51
18,愛知県,2023-03-24,ミカド稲沢店,マキシマムスロッター,B,234.0,パチンコ店長のホール攻略,未調査,2023-03-16 03:56:42
21,愛知県,2023-03-24,マルハン上小田井駅前店,光るん×スロッター,・,198.0,パチンコ店長のホール攻略,未調査,2023-03-16 03:56:41
22,愛知県,2023-03-24,ミカド一宮本店,サプライズスロッター,A,488.0,パチンコ店長のホール攻略,未調査,2023-03-16 03:56:55


In [27]:

def recommend_image(write_image_context, length,read_image_path,save_image_jpg):

    image = Image.open(read_image_path)
    draw = ImageDraw.Draw(image)
    # フォントを指定
    font_path = r"font\LightNovelPOPv2.otf"
    font = ImageFont.truetype(font_path, size=40)

    # 文字を描く
    # 最初の(0,0)は文字の描画を開f始する座標位置です　もちろん、(10,10)などでもOK
    # fillはRGBで文字の色を決めています
    draw.multiline_text((55, 40), write_image_context, fill=(255, 255, 255), font=font, spacing=10, stroke_width=5, stroke_fill=(55, 55, 55))

    # 画像を保存する
    image.save(save_image_jpg)
    print('length', length)
    length_croped = 110 + length * 66  # 500
    im = Image.open(save_image_jpg)
    im_crop = im.crop((0, 0, 1000, length_croped))
    im_crop.save(save_image_jpg, quality=100)

In [28]:
read_image_path = r"image\image_source\eva_board.jpg"
prefecture_name = '愛知県'
create_image_number = 0
parlar_name_count = 0
schedule_text = ''
last_time_parlar_name = ''

for i,row in merged_syuzai_pledge_df.iterrows():
    #print(row)
    print(row['店舗名'],last_time_parlar_name )
    if last_time_parlar_name != row['店舗名']:
        schedule_text += '\n◆' +  row['店舗名'] + '\n'
        parlar_name_count += 1
        #print(parlar_name_count)
    else:
        pass


    if row['取材名'].startswith(tuple(['旧イベ'])):
        schedule_text += '　☆' + row['取材名']  + '\n'
        last_time_parlar_name = row['店舗名']
        continue
    else:
        schedule_text += '　☆' + row['取材名'] + ' 【' + row['媒体名'] + '】' + '\n'
        

    if (str(row['公約内容']) == 'nan') or (row['公約内容'] == None) or (row['公約内容'] == ''):
        schedule_text += '       ┗' + '未調査' + '\n'
        
    else:
        schedule_text += '       ┗' + row['公約内容'] + '\n'

    last_time_parlar_name = row['店舗名']
    
    #print(parlar_name_count)
    if (parlar_name_count > 9 ) or merged_syuzai_pledge_df.index[-1] == i :
        #print('処理開始')
        create_image_number  += 1
        title = f'{prefecture_name} {twitter.tomorrow_date_stinrg_jp} 明日のスロットイベントまとめ{create_image_number}\n'
        input_image_text : str = title + schedule_text
        text__row_length = input_image_text.count('\n')
        #print('length', text__row_length )
        print(input_image_text)
        save_image_jpg = fr"image\temp_image\eva_board_{prefecture_name}_{target_date_str_sql}_{create_image_number}.jpg"
        recommend_image(input_image_text, text__row_length ,read_image_path,save_image_jpg)
        schedule_text = ''
        last_time_parlar_name = ''
        parlar_name_count = 0

print(input_image_text)

キクヤ春日井店 
キング観光 サウザンド栄若宮大通店 キクヤ春日井店
KEIZ 尾張旭店 キング観光 サウザンド栄若宮大通店
プレイランドキャッスル熱田店 KEIZ 尾張旭店
キング観光サウザンド名古屋駅柳橋店 プレイランドキャッスル熱田店
マルハン新守山駅前店 キング観光サウザンド名古屋駅柳橋店
メガコンコルド1020刈谷知立店 マルハン新守山駅前店
ミカド稲沢店 メガコンコルド1020刈谷知立店
マルハン上小田井駅前店 ミカド稲沢店
ミカド一宮本店 マルハン上小田井駅前店
愛知県 3/24(金) 明日のスロットイベントまとめ1

◆キクヤ春日井店
　☆シーサーペントOVER 【アツ姫】
       ┗未調査

◆キング観光 サウザンド栄若宮大通店
　☆オフミー開催(BBオフ会) 【オフミー】
       ┗未調査

◆KEIZ 尾張旭店
　☆まなみん来店 【その他取材・企画】
       ┗未調査

◆プレイランドキャッスル熱田店
　☆シーサ。オフミー取材 【オフミー】
       ┗未調査

◆キング観光サウザンド名古屋駅柳橋店
　☆スロパチステーション潜入取材 【スロパチステーション】
       ┗未調査

◆マルハン新守山駅前店
　☆光るん×スロッター 【パチンコ店長のホール攻略】
       ┗未調査

◆メガコンコルド1020刈谷知立店
　☆ズバスロ 【あかだ主任のガチンコ…】
       ┗未調査

◆ミカド稲沢店
　☆マキシマムスロッター 【パチンコ店長のホール攻略】
       ┗未調査

◆マルハン上小田井駅前店
　☆光るん×スロッター 【パチンコ店長のホール攻略】
       ┗未調査

◆ミカド一宮本店
　☆サプライズスロッター 【パチンコ店長のホール攻略】
       ┗未調査

length 41
MEGAコンコルド1280稲沢店 
MEGAコンコルド1280稲沢店 MEGAコンコルド1280稲沢店
プレイランドキャッスル尾頭橋店 MEGAコンコルド1280稲沢店
プレイランドキャッスル記念橋南店 プレイランドキャッスル尾頭橋店
マルハン中川店 プレイランドキャッスル記念橋南店
がちゃぽん蟹江店 マルハン中川店
名宝刈谷店 がちゃぽん蟹江店
メガコンコルド800春日井店+ 名宝刈谷店
ラッキープラザ名古屋西インター

In [13]:
html_table_df = merged_syuzai_pledge_df[['イベント日','店舗名','取材名','媒体名','公約内容']]
html_table_df['イベント日'] = twitter.tomorrow_date_stinrg_jp
html_table_df = html_table_df[['イベント日','店舗名','取材名','媒体名','公約内容']]
print(html_table_df.to_html(index=False))

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>イベント日</th>
      <th>店舗名</th>
      <th>取材名</th>
      <th>媒体名</th>
      <th>公約内容</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>3/24(金)</td>
      <td>キクヤ春日井店</td>
      <td>シーサーペントOVER</td>
      <td>アツ姫</td>
      <td></td>
    </tr>
    <tr>
      <td>3/24(金)</td>
      <td>キング観光 サウザンド栄若宮大通店</td>
      <td>オフミー開催(BBオフ会)</td>
      <td>オフミー</td>
      <td></td>
    </tr>
    <tr>
      <td>3/24(金)</td>
      <td>KEIZ 尾張旭店</td>
      <td>まなみん来店</td>
      <td>その他取材・企画</td>
      <td></td>
    </tr>
    <tr>
      <td>3/24(金)</td>
      <td>プレイランドキャッスル熱田店</td>
      <td>シーサ。オフミー取材</td>
      <td>オフミー</td>
      <td></td>
    </tr>
    <tr>
      <td>3/24(金)</td>
      <td>キング観光サウザンド名古屋駅柳橋店</td>
      <td>スロパチステーション潜入取材</td>
      <td>スロパチステーション</td>
      <td></td>
    </tr>
    <tr>
      <td>3/24(金)</td>
      <td>マルハン新守山駅前店</td>
      <td>光るん×スロッター</td>
      <td>パチンコ店長のホール攻略</td>


C:\Users\tsc95\AppData\Local\Temp\ipykernel_48568\3955047594.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  html_table_df['イベント日'] = twitter.tomorrow_date_stinrg_jp
